# HHL
In the following notebook we are going to let a real quantum device solve a linear system $A\vec x = \vec b$ .


As usual, lets get set up.
We import the classes and functions from qiskit as well as some custom functions that will help us adding special gates to the circuit

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.tools.qi.pauli import Pauli
from qiskit_aqua import Operator, QuantumAlgorithm, AlgorithmError
from qiskit_aqua import get_initial_state_instance, get_iqft_instance

from qiskit.tools.visualization._circuit_visualization import matplotlib_circuit_drawer

import numpy as np


from qpe import QPE
from fix_rotation import add_eigenvalue_inversion, add_measurement_gates, generate_matrix

In [2]:
N = 2
N_ev = 3
N_sv = int(np.log2(N))
measure = True
backend = "local_qasm_simulator"
print(N,N_ev,N_sv)

2 3 1


The following routine will generate a matrix by adding multiple Pauli matrices.
We restrict our linear system to be real valued (i.e. not complex) and make use of the Pauli matrices $\sigma_{I}=\begin{pmatrix}1 & 0 \\ 0 & 1 \end{pmatrix}$, 
$\sigma_{x}=\begin{pmatrix}0 & 1 \\ 1 & 0 \end{pmatrix}$, $\sigma_{z}=\begin{pmatrix}1 & 0 \\ 0 & -1 \end{pmatrix}$
Now we have a matrix $A=\sum_i \alpha_{i} \sigma_{l_{i}}\ l_{i}\in \{I,x,z\}$ that is going to specify our linear system!

## State preparation and QPE

The HHL algorithm starts off with the preparation of $\vec b$ in one register followed by the Quantum Phase Estimation that decomposes $\vec b$ into eigenvectors of $A$ and prepares the eigenvalues in a second register.

In [20]:

matrix =generate_matrix()#np.array([[2, -1 ],[-1, 2]])#generate_matrix()

w, v = np.linalg.eigh(matrix) 

print(matrix)
print("Eigenvalues:", w)

invec = sum([v[:,i] for i in range(N)])
invec /= np.sqrt(invec.dot(invec.conj()))

invec =  [1,0]#1/np.sqrt(2)*np.array([1,1])#[1,0]#

params = {
    'algorithm': {
            'name': 'QPE',
            'num_ancillae':3,
            'num_time_slices': 1,
            'expansion_mode': 'trotter',
            'expansion_order': 1,
            'hermitian_matrix': True,
            'negative_evals': False,
            'backend' : "local_qasm_simulator",
            'evo_time': 2*np.pi/16,
            #'use_basis_gates': False,
    },
    "iqft": {
        "name": "STANDARD"
    },
    "initial_state": {
        "name": "CUSTOM",
        "state_vector": invec#[1/2**0.5,1/2**0.5]
    }
}




[[ 2 -1]
 [-1  2]]
Eigenvalues: [1. 3.]


In [ ]:
qpe = QPE()
qpe.init_params(params, matrix)

qc = qpe._setup_qpe(measure=False)
#matplotlib_circuit_drawer(qc)

Take a look at above circuit. The gate(s) in front of the first U gate that is applied to quibit 0 of the topmost register, prepare us the initial state vector $\vec b$ .
Following, so called Hamiltonian simulation techniques is used to apply the operator $\exp{i 2\pi A/t}$ to our register containing the state vector. 


## Inversion and rotation
The last step before obtaining our measurement is to invert $A$. We have $\vec b$ represented in the basis of eigenvectors of A so simply using the inverse eigenvalues gives us $A^{-1}$ . 
Given that this transformation of the eigenvalues from $\lambda$ to $\lambda^{-1}$ is not unitary, we have a probability of failing this step. Unfortunately we do not have direct access to the state of the qubits and we can not "measure" if the inversion suceeded. Instead we will perform a rotation of $\arcsin{\frac{C}{\lambda}}$ on a "control" qubit.

In [5]:

add_eigenvalue_inversion(qc)
#matplotlib_circuit_drawer(qc)

In [6]:
qc += qpe._construct_inverse()
#matplotlib_circuit_drawer(qc)

Now we add some measurements to our circuit. The qubit storing the solution vector and the control qubit are read out into classical registers

In [7]:
add_measurement_gates(qc)

#matplotlib_circuit_drawer(qc)

We are ready to run our HHL program! 

In [8]:
result = execute(qc, backend=backend, shots=5000).result()
print(result.get_counts())

{'0 0': 12, '0 1': 64, '1 0': 4760, '1 1': 164}


In [9]:
N0 = result.get_counts(qc)['1 1']
N1 = result.get_counts(qc)['0 1']
print(N0/N1)



2.5625


The ratio of N0 and N1, i.e. the probability of measuring 1, 0 respectively in the solution vector qubit, gives us the relative scale of the two entries in the vector.

Lets repeat our process several times to obtain better statistics... 

In [21]:
N0 = 0
N1 = 0
for i in range(10):
    qpe = QPE()


    qpe.init_params(params, matrix)

    qc = qpe._setup_qpe(measure=False)
    add_eigenvalue_inversion(qc)
    qc += qpe._construct_inverse()
    add_measurement_gates(qc)
    result = execute(qc, backend=backend, shots=8100).result()
    print(result.get_counts(qc))
    N0 = N0 + result.get_counts(qc)['1 1']
    N1 = N1 + result.get_counts(qc)['0 1']

    

QPE circuit qasm length is roughly 51.
{'0 0': 7158, '0 1': 667, '1 0': 108, '1 1': 167}
QPE circuit qasm length is roughly 51.
{'0 0': 7161, '0 1': 665, '1 0': 95, '1 1': 179}
QPE circuit qasm length is roughly 51.
{'0 0': 7137, '0 1': 665, '1 0': 92, '1 1': 206}
QPE circuit qasm length is roughly 51.
{'0 0': 7164, '0 1': 661, '1 0': 88, '1 1': 187}
QPE circuit qasm length is roughly 51.
{'0 0': 7155, '0 1': 704, '1 0': 87, '1 1': 154}
QPE circuit qasm length is roughly 51.
{'0 0': 7147, '0 1': 672, '1 0': 90, '1 1': 191}
QPE circuit qasm length is roughly 51.
{'0 0': 7169, '0 1': 697, '1 0': 78, '1 1': 156}
QPE circuit qasm length is roughly 51.
{'0 0': 7184, '0 1': 679, '1 0': 76, '1 1': 161}
QPE circuit qasm length is roughly 51.
{'0 0': 7098, '0 1': 747, '1 0': 94, '1 1': 161}
QPE circuit qasm length is roughly 51.
{'0 0': 7133, '0 1': 686, '1 0': 97, '1 1': 184}


In [22]:
N1/N0

3.9192439862542954

array([0.89868383, 0.10131617])

array([0.33333333, 0.66666667])